In [1]:
from utils.imports import *

('/Volumes/solo/ali/Data/train_cls_cube_36/train/true/', u'maked')
('/Volumes/solo/ali/Data/train_cls_cube_36/train/false/', u'maked')


Using TensorFlow backend.


In [2]:
def create_cls_sample(df_anno,df_pred,img_file,data_path,output_true,output_false):
    mini_df_anno = df_anno[df_anno["file"]==img_file] 
    mini_df_pred = df_pred[df_pred["file"]==img_file]
    if mini_df_anno.shape[0]>0:
        patient_id = img_file[:-9]
        img_array = np.load(data_path + img_file)
        img_array = normalize(img_array)
        pos_annos = pd.read_csv(data_path + img_file[:-9] + '_annos_pos.csv')
        origin = np.array([pos_annos.loc[0]['origin_x'],pos_annos.loc[0]['origin_y'],pos_annos.loc[0]['origin_z']]) 
        spacing = np.array([pos_annos.loc[0]['spacing_x'],pos_annos.loc[0]['spacing_y'],pos_annos.loc[0]['spacing_z']]) 
        
        for node_idx1, cur_row1 in mini_df_anno.iterrows():       
            node_x = cur_row1["coordX"]
            node_y = cur_row1["coordY"]
            node_z = cur_row1["coordZ"]
            diam = cur_row1["diameter_mm"]
            center = np.array([node_x, node_y, node_z])   
            v_center = np.rint(np.absolute(center-origin)/spacing) 
            
            new_x = int(v_center[0])
            new_y = int(v_center[1])
            new_z = int(v_center[2])
            trainX_1 =  img_array[new_z - 16 : new_z + 16,
                                  new_y - 16 : new_y + 16,
                                  new_x - 16 : new_x + 16] 
            
            if trainX_1.shape == (32,32,32):
                np.save(output_true + str(patient_id)+'_'+str(node_idx1)+str('_0')+ '.npy',trainX_1)
                np.save(output_true + str(patient_id)+'_'+str(node_idx1)+str('_1')+ '.npy',np.fliplr(trainX_1))
                np.save(output_true + str(patient_id)+'_'+str(node_idx1)+str('_2')+ '.npy',np.flipud(trainX_1))
            
            new_z1 = new_z + random.choice([-3,-2,-1,1,2,3])
            new_y1 = new_y + random.choice([-3,-2,-1,1,2,3])
            new_x1 = new_x + random.choice([-3,-2,-1,1,2,3])
            
            trainX_2 =  img_array[new_z1 - 16 : new_z1 + 16,
                                  new_y1 - 16 : new_y1 + 16,
                                  new_x1 - 16 : new_x1 + 16] 
            
            if trainX_2.shape == (32,32,32):
                np.save(output_true + str(patient_id)+'_'+str(node_idx1)+str('_3')+ '.npy',trainX_2)
                np.save(output_true + str(patient_id)+'_'+str(node_idx1)+str('_4')+ '.npy',np.fliplr(trainX_2))
                np.save(output_true + str(patient_id)+'_'+str(node_idx1)+str('_5')+ '.npy',np.flipud(trainX_2))

                
            new_z2 = new_z + random.choice([-3,-2,-1,1,2,3])
            new_y2 = new_y + random.choice([-3,-2,-1,1,2,3])
            new_x2 = new_x + random.choice([-3,-2,-1,1,2,3])    
            trainX_3 =  img_array[new_z2 - 16 : new_z2 + 16,
                                  new_y2 - 16 : new_y2 + 16,
                                  new_x2 - 16 : new_x2 + 16] 
            
            if trainX_3.shape == (32,32,32):
                np.save(output_true + str(patient_id)+'_'+str(node_idx1)+str('_6')+ '.npy',trainX_3)
                np.save(output_true + str(patient_id)+'_'+str(node_idx1)+str('_7')+ '.npy',np.fliplr(trainX_3))
                np.save(output_true + str(patient_id)+'_'+str(node_idx1)+str('_8')+ '.npy',np.flipud(trainX_3))

                        
                  

        for node_idx2, cur_row2 in mini_df_pred.iterrows():       
            node_x = cur_row2["coordX"]
            node_y = cur_row2["coordY"]
            node_z = cur_row2["coordZ"]
            diam = cur_row2["diameter_mm"]
            center = np.array([node_x, node_y, node_z])   
            v_center = np.rint(np.absolute(center-origin)/spacing)  
            
            
            new_x = int(v_center[0])
            new_y = int(v_center[1])
            new_z = int(v_center[2])
            
            trainX =   img_array[new_z - 16 : new_z + 16,
                                 new_y - 16 : new_y + 16,
                                 new_x - 16 : new_x + 16] 
    
            if trainX.shape == (32,32,32):
                np.save(output_false + str(patient_id)+'_'+str(node_idx2)+str('_0') + '.npy',trainX)
    return


In [3]:
csv_path = PATH['annotations_train']
output_true = PATH['cls_train_cube_30_true']
output_false = PATH['cls_train_cube_30_false']
pred_csv_path = PATH['model_train_pred']
data_path = PATH['model_train_pred']
anno_csv_new = pd.read_csv(pred_csv_path + "anno_true_final.csv")
pred_csv_new = pd.read_csv(pred_csv_path + "anno_false_final.csv")
#pred_csv_new = pd.read_csv(pred_csv_path + "anno_false_final.csv")

In [4]:
patients = [x for x in os.listdir(data_path) if 'orig.npy' in x]

In [5]:
anno_csv_new["file"] = anno_csv_new["seriesuid"].map(lambda file_name: get_filename(patients, file_name))
anno_csv_new = anno_csv_new.dropna()
pred_csv_new["file"] = pred_csv_new["seriesuid"].map(lambda file_name: get_filename(patients, file_name))
pred_csv_new = pred_csv_new.dropna()

In [6]:
Parallel(n_jobs=-1)(delayed(create_cls_sample)(anno_csv_new,pred_csv_new,patient,data_path,output_true,output_false) for patient in tqdm(sorted(patients)))

 40%|███▉      | 319/800 [22:46<26:56,  3.36s/it]  

KeyboardInterrupt: 